In [16]:
import os
import sys
import copy
import pickle
import pandas as pd

import numpy as np
import rmgpy.chemkin

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
basedir = '/home/moon/autoscience/autoscience/butane/models/rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)




# Kinetics

In [4]:
# get most uncertain reactions
covariance_file = '/home/moon/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)
uncorrelated_uncertainties = np.diagonal(Sigma_k)
uncorrelated_uncertainties = np.diagonal(Sigma_k)
reaction_indices = np.arange(0, len(reaction_list))
reaction_uncertainty_order = [x for _,x in sorted(zip(uncorrelated_uncertainties, reaction_indices))][::-1]

In [5]:
# get most sensitive reactions
base_delay = 0.14857535
reaction_delay_file = '/home/moon/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/reaction_ignition_delays_raw.npy'
reaction_delays_raw = np.load(reaction_delay_file)
reaction_sensitivities = ((reaction_delays_raw - base_delay) / base_delay) / 0.1

reaction_sensitivity_order = [x for _,x in sorted(zip(np.abs(reaction_sensitivities), reaction_indices))][::-1]


In [6]:
# Calculate Improvement Score
DFT_error = 3.0
improvement_score = np.abs(reaction_sensitivities) * (uncorrelated_uncertainties - DFT_error)
improvement_score_order = [x for _,x in sorted(zip(improvement_score, reaction_indices))][::-1]


In [7]:
print('Top Uncertain Reactions')
print('i\tDelta\tReaction\tSensitivity\tImprovement Score')
for i in range(0, 10):
    rmg_index = reaction_uncertainty_order[i]
    print(rmg_index, '\t', np.round(uncorrelated_uncertainties[rmg_index], 3),
          '\t', reaction_list[rmg_index],
          '\t', f'{reaction_sensitivities[rmg_index]:.3e}',
          '\t', f'{improvement_score[rmg_index]:.3e}')
    
    
print()
for i in range(0, 10):
    rmg_index = reaction_uncertainty_order[i]
    print(reaction_list[rmg_index].family)

Top Uncertain Reactions
i	Delta	Reaction	Sensitivity	Improvement Score
655 	 13.479 	 HO2(16) + [CH2]C[CH]C(748) <=> O2(2) + PC4H9(182) 	 -3.229e-07 	 3.384e-06
1003 	 13.408 	 HO2(16) + C4H71-4(191) <=> O2(2) + C4H8-1(188) 	 -3.095e-07 	 3.222e-06
985 	 13.408 	 HO2(16) + [CH2]C[O](703) <=> O2(2) + C2H5O(49) 	 -2.786e-07 	 2.899e-06
692 	 13.408 	 HO2(16) + [CH2]CC(C)[O](766) <=> O2(2) + SC4H9O(217) 	 -2.575e-07 	 2.680e-06
650 	 13.408 	 HO2(16) + [CH2]C[CH]C(748) <=> O2(2) + SC4H9(183) 	 -2.926e-07 	 3.045e-06
604 	 11.92 	 C2H(4) + C3H6(12) <=> C2H2(25) + C3H5-A(94) 	 -2.813e-07 	 2.509e-06
585 	 11.92 	 CH2(23) + C3H6(12) <=> CH3(18) + C3H5-A(94) 	 -3.162e-07 	 2.820e-06
773 	 11.238 	 CH3CO(20) + C4H8-1(188) <=> CH3CHO(35) + C4H71-3(190) 	 -3.042e-07 	 2.506e-06
756 	 11.041 	 C2H(4) + C4H8-1(188) <=> C2H2(25) + C4H71-3(190) 	 -2.321e-07 	 1.867e-06
753 	 11.041 	 CH2(23) + C4H8-1(188) <=> CH3(18) + C4H71-3(190) 	 -3.060e-07 	 2.461e-06

H_Abstraction
H_Abstraction
H_Abstraction


In [8]:
print('Top Sensitive Reactions')
print('i\tDelta\tReaction\tSensitivity')
for i in range(0, 10):
    rmg_index = reaction_sensitivity_order[i]
    print(rmg_index, '\t', np.round(uncorrelated_uncertainties[rmg_index], 3),
          '\t', reaction_list[rmg_index],
          '\t', f'{reaction_sensitivities[rmg_index]:.3e}',
          '\t', f'{improvement_score[rmg_index]:.3e}')

Top Sensitive Reactions
i	Delta	Reaction	Sensitivity
288 	 0.5 	 OH(15) + butane(1) <=> H2O(8) + PC4H9(182) 	 -1.314e+00 	 -3.285e+00
247 	 0.5 	 OH(15) + butane(1) <=> H2O(8) + SC4H9(183) 	 1.222e+00 	 -3.055e+00
279 	 0.5 	 SC4H9O2(186) <=> HO2(16) + C4H8-2(189) 	 6.481e-01 	 -1.620e+00
323 	 0.5 	 O2(2) + C4H8OOH1-3(219) <=> C4H8OOH1-3O2(225) 	 -6.427e-01 	 -1.607e+00
281 	 0.5 	 O2(2) + C4H8OOH2-4(223) <=> C4H8OOH2-4O2(229) 	 -6.120e-01 	 -1.530e+00
316 	 0.5 	 PC4H9O2(184) <=> HO2(16) + C4H8-1(188) 	 5.423e-01 	 -1.356e+00
249 	 0.5 	 HO2(16) + butane(1) <=> H2O2(17) + SC4H9(183) 	 -4.697e-01 	 -1.174e+00
449 	 0.5 	 C4H8OOH1-3(219) <=> OH(15) + C4H8O1-3(214) 	 4.667e-01 	 -1.167e+00
317 	 0.5 	 SC4H9O2(186) <=> HO2(16) + C4H8-1(188) 	 4.153e-01 	 -1.038e+00
326 	 0.5 	 C4H8OOH1-3O2(225) <=> C4H71-1,3OOH(238) 	 -3.459e-01 	 -8.648e-01


In [9]:
print('Top Improvement Score Reactions')
print('i\tDelta\tReaction\tSensitivity')
for i in range(0, 10):
    rmg_index = improvement_score_order[i]
    print(rmg_index, '\t', np.round(uncorrelated_uncertainties[rmg_index], 3),
          '\t', reaction_list[rmg_index],
          '\t', f'{reaction_sensitivities[rmg_index]:.3e}',
          '\t', f'{improvement_score[rmg_index]:.3e}')

Top Improvement Score Reactions
i	Delta	Reaction	Sensitivity
915 	 10.688 	 O2(2) + CC(CC[O])OO(777) <=> HO2(16) + NC4KET13(252) 	 -1.025e-03 	 7.883e-03
749 	 11.041 	 HCO(19) + C4H8-1(188) <=> CH2O(9) + C4H71-3(190) 	 5.078e-04 	 4.083e-03
324 	 3.617 	 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225) 	 -4.290e-03 	 2.649e-03
419 	 10.688 	 HO2(16) + CH2CHO(21) <=> H2O2(17) + CH2CO(24) 	 2.961e-04 	 2.276e-03
1814 	 10.688 	 O2(2) + [CH]1CCOOC1(5425) <=> HO2(16) + C1=COOCC1(5951) 	 -2.031e-04 	 1.561e-03
1287 	 3.93 	 C3KET21(73) + SC4H9(183) <=> CH3COCH2O2(72) + butane(1) 	 -1.563e-03 	 1.453e-03
748 	 3.229 	 HCO(19) + C4H8-2(189) <=> CH2O(9) + C4H71-3(190) 	 2.681e-03 	 6.130e-04
1288 	 4.253 	 C3KET21(73) + PC4H9(182) <=> CH3COCH2O2(72) + butane(1) 	 -4.743e-04 	 5.941e-04
370 	 4.253 	 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1) 	 3.559e-04 	 4.459e-04
1103 	 10.688 	 CH3O2(45) + CH2CHO(21) <=> CH3O2H(46) + CH2CO(24) 	 2.013e-05 	 1.547e-04


# Thermodynamics

In [10]:
# get most uncertain species
species_uncertainty = '/home/moon/autoscience/autoscience/paper/models/base_rmg/thermo_uncertainty.npy'
species_indices = np.arange(0, len(reaction_list))
uncorrelated_species_uncertainties = np.load(species_uncertainty)
species_uncertainty_order = [x for _,x in sorted(zip(uncorrelated_species_uncertainties, species_indices))][::-1]

In [11]:
# get most sensitive species
base_delay = 0.14857535
species_delay_file = '/home/moon/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/species_ignition_delays_raw.npy'
species_delays_raw = np.load(species_delay_file)
species_sensitivities = ((species_delays_raw - base_delay) / base_delay) / 0.1

species_sensitivity_order = [x for _,x in sorted(zip(np.abs(species_sensitivities), species_indices))][::-1]


In [12]:
# Calculate Improvement Score
DFT_error = 3.0
species_improvement_score = np.abs(species_sensitivities) * (uncorrelated_species_uncertainties - DFT_error)
species_improvement_score_order = [x for _,x in sorted(zip(species_improvement_score, species_indices))][::-1]

In [13]:
print('Top Uncertain Species')
print('i\tSpecies\tUncertainty\tSensitivity\tImprovement Score')
for i in range(0, 20):
    rmg_sp_index = species_uncertainty_order[i]
    print(rmg_sp_index, '\t', species_list[rmg_sp_index], '\t',
          np.round(uncorrelated_species_uncertainties[rmg_sp_index], 3),
          '\t', f'{species_sensitivities[rmg_sp_index]:.3e}',
          '\t', f'{species_improvement_score[rmg_sp_index]:.3e}')
    
print()
for i in range(0, 20):
    rmg_sp_index = species_uncertainty_order[i]
    print(species_list[rmg_sp_index].thermo.comment)

Top Uncertain Species
i	Species	Uncertainty	Sensitivity	Improvement Score
93 	 [CH2]C=C[CH2](2534) 	 3.4 	 -2.522e-07 	 1.009e-07
91 	 [O]C[O](988) 	 3.4 	 -3.067e-07 	 1.227e-07
109 	 [O]OC1CCOOC1(9510) 	 3.3 	 -6.172e-05 	 1.852e-05
82 	 [CH2]CC(C)[O](766) 	 3.2 	 -2.838e-07 	 5.675e-08
81 	 [CH2]C[CH]C(748) 	 3.2 	 -7.727e-07 	 1.545e-07
80 	 [CH2]C[O](703) 	 3.2 	 2.254e-06 	 4.508e-07
100 	 C=CCCO[O](4407) 	 3.1 	 -1.677e-06 	 1.677e-07
99 	 [CH2]C1CC1(4343) 	 3.1 	 2.941e-05 	 2.941e-06
98 	 CC=CCO[O](4244) 	 3.1 	 -3.493e-06 	 3.493e-07
97 	 C=CC(C)O[O](4243) 	 3.1 	 -9.326e-07 	 9.326e-08
90 	 [O]OCC(=O)COO(928) 	 3.1 	 1.160e-03 	 1.160e-04
87 	 [CH2]C(=O)COO(911) 	 3.1 	 2.025e-01 	 2.025e-02
105 	 [CH]1CCOOC1(5425) 	 2.9 	 1.587e-03 	 -1.587e-04
85 	 CC(CCOO)OO(787) 	 2.7 	 1.742e+00 	 -5.227e-01
101 	 [C]1CCC1(4880) 	 2.6 	 -3.580e-07 	 -1.432e-07
103 	 OCCC1CO1(5151) 	 2.4 	 -1.164e-03 	 -6.982e-04
92 	 C1=CCC1(2530) 	 2.4 	 -8.035e-06 	 -4.821e-06
106 	 [O]C=CCC[O](5753) 

In [14]:
print('Top Sensitive Species')
print('i\tSpecies\tUncertainty\tSensitivity\tImprovement Score')
for i in range(0, 10):
    rmg_sp_index = species_sensitivity_order[i]
    print(rmg_sp_index, '\t', species_list[rmg_sp_index], '\t',
          np.round(uncorrelated_species_uncertainties[rmg_sp_index], 3),
          '\t', f'{species_sensitivities[rmg_sp_index]:.3e}',
          '\t', f'{species_improvement_score[rmg_sp_index]:.3e}')

Top Sensitive Species
i	Species	Uncertainty	Sensitivity	Improvement Score
59 	 SC4H9O2(186) 	 1.5 	 1.026e+01 	 -1.539e+01
58 	 PC4H9O2(184) 	 1.5 	 1.014e+01 	 -1.521e+01
73 	 C4H8OOH1-3O2(225) 	 1.5 	 -4.121e+00 	 -6.182e+00
72 	 C4H8OOH2-4(223) 	 1.5 	 -3.734e+00 	 -5.601e+00
74 	 C4H8OOH2-4O2(229) 	 1.5 	 -3.594e+00 	 -5.391e+00
85 	 CC(CCOO)OO(787) 	 2.7 	 1.742e+00 	 -5.227e-01
46 	 C3KET21(73) 	 1.5 	 -1.214e+00 	 -1.821e+00
5 	 O2(2) 	 1.5 	 7.474e-01 	 -1.121e+00
32 	 C2H5(33) 	 1.5 	 -3.995e-01 	 -5.993e-01
21 	 CH3(18) 	 1.5 	 -3.828e-01 	 -5.742e-01


In [15]:
print('Top Improvement Species')
print('i\tSpecies\tUncertainty\tSensitivity\tImprovement Score')
for i in range(0, 10):
    rmg_sp_index = species_improvement_score_order[i]
    print(rmg_sp_index, '\t', species_list[rmg_sp_index], '\t',
          np.round(uncorrelated_species_uncertainties[rmg_sp_index], 3),
          '\t', f'{species_sensitivities[rmg_sp_index]:.3e}',
          '\t', f'{species_improvement_score[rmg_sp_index]:.3e}')

Top Improvement Species
i	Species	Uncertainty	Sensitivity	Improvement Score
87 	 [CH2]C(=O)COO(911) 	 3.1 	 2.025e-01 	 2.025e-02
90 	 [O]OCC(=O)COO(928) 	 3.1 	 1.160e-03 	 1.160e-04
109 	 [O]OC1CCOOC1(9510) 	 3.3 	 -6.172e-05 	 1.852e-05
99 	 [CH2]C1CC1(4343) 	 3.1 	 2.941e-05 	 2.941e-06
80 	 [CH2]C[O](703) 	 3.2 	 2.254e-06 	 4.508e-07
98 	 CC=CCO[O](4244) 	 3.1 	 -3.493e-06 	 3.493e-07
100 	 C=CCCO[O](4407) 	 3.1 	 -1.677e-06 	 1.677e-07
81 	 [CH2]C[CH]C(748) 	 3.2 	 -7.727e-07 	 1.545e-07
91 	 [O]C[O](988) 	 3.4 	 -3.067e-07 	 1.227e-07
93 	 [CH2]C=C[CH2](2534) 	 3.4 	 -2.522e-07 	 1.009e-07


In [17]:
species_list[0].smiles

'N#N'

In [23]:
# Now combine them into top everything
total_df = pd.DataFrame()


# indices = []
rmg_indices = []
names = []
entry_types = []
sensitivities = []
uncertainties = []
improvement_scores = []

for i in range(0, len(species_list)):
    rmg_indices.append(i)
    names.append(species_list[i].smiles)
    entry_types.append('species')
    sensitivities.append(species_sensitivities[i])
    uncertainties.append(uncorrelated_species_uncertainties[i])
    improvement_scores.append(species_improvement_score[i])
    

for i in range(0, len(reaction_list)):
    rmg_indices.append(i)
    names.append(str(reaction_list[i]))
    entry_types.append('reaction')
    sensitivities.append(reaction_sensitivities[i])
    uncertainties.append(uncorrelated_uncertainties[i])
    improvement_scores.append(improvement_score[i])
    

total_df['rmg_index'] = rmg_indices
total_df['name'] = names
total_df['entry_type'] = entry_types
total_df['sensitivity'] = sensitivities
total_df['uncertaintie'] = uncertainties
total_df['improvement_score'] = improvement_scores
total_df.to_csv('mechanism_summary.csv')



In [26]:
# read the dataframe

df = pd.read_csv('mechanism_summary.csv')
if 'Unnamed: 0' in df .keys():  
    df  = df .drop(columns=['Unnamed: 0'])
print(df)


      rmg_index                                         name entry_type  \
0             0                                          N#N    species   
1             1                                         [Ar]    species   
2             2                                         [He]    species   
3             3                                         [Ne]    species   
4             4                                         CCCC    species   
...         ...                                          ...        ...   
1927       1817  [O]C=CCC[O](5753) <=> CH2O(9) + C2H3CHO(74)   reaction   
1928       1818        [O]C=CCC[O](5753) <=> C1=COOCC1(5951)   reaction   
1929       1819    O=CC1CCO1(5761) <=> CH2O(9) + C2H3CHO(74)   reaction   
1930       1820          C1=COOCC1(5951) <=> O=CC1CCO1(5761)   reaction   
1931       1821        [O]C=CCC[O](5753) <=> O=CC1CCO1(5761)   reaction   

       sensitivity  uncertaintie  improvement_score  
0    -2.822246e-07           1.5      -4.2333